<a href="https://colab.research.google.com/github/RafaKuniyoshi/mask-detection-yolov4/blob/main/Mask_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparação do ambiente e download das bibliotecas necessárias

Primerio de tudo, precisamos importar os pacotes necessários para realizar as detecções

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import xml.etree.ElementTree as ET 
import cv2
import glob
from sklearn.model_selection import train_test_split

Agora será necessário instalar o pacote do kaggle para fazer o download dos dados. Será necessário forçar a versão 1.5.8


In [ ]:
!pip install -U -q kaggle==1.5.8

     |████████████████████████████████| 59 kB 3.8 MB/s 


Verificação que estamos utilizando um runtime com GPU

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Verificando a quantidade de memória RAM disponível

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


Agora vamos montar o disco do drive para salvar o dataset e outras informações vindas do YoloV4

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Nessa parte estamos criando um link simbólico para o drive e criando a pasta onde vamos salvar o dataset e algumas outras informações

In [ ]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

%cd drive/MyDrive/
%mkdir faceMaskYolo
%cd faceMaskYolo/
%mkdir backup

/content/drive/MyDrive


Vamos fazer o upload da chave do Kaggle para fazer o download do dataset

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 64 bytes


Vamos copiar os dados do Kaggle utilizando a api que instalamos

In [ ]:
!kaggle datasets download -d andrewmvd/face-mask-detection
!mkdir face-mask-detection
!unzip face-mask-detection.zip -d face-mask-detection

Voltamos para a raiz do projeto

In [ ]:
%cd /content
%ls


/content
drive/  sample_data/


Agora preciso copiar os arquivos da nossa rede neural. Vamos utilizar o darknet que apresenta resultados incríveis.
Essa parte da intalação está exatamente igual ao que pode ser encontrado no github do autor do Yolov4

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15412, done.
remote: Total 15412 (delta 0), reused 0 (delta 0), pack-reused 15412
Receiving objects: 100% (15412/15412), 14.02 MiB | 23.89 MiB/s, done.
Resolving deltas: 100% (10356/10356), done.


Agora é necessário fazer algumas alterações no arquivo Makefile para liberar o usu do OPENCV e GPU

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

/content/darknet


Verificamos se estamos usando o CUDA

In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


Hora de dar um build no darknet

In [ ]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

# Tratamento do dataset e criação das funções necessárias para o treinamento

Vamos definir a função convert, que será responsável por passar as anotações do formato PASCAL para o formato esperado pelo YOLOv4

In [ ]:
def convert(size: tuple, box: list): 
    largura_relativa = 1./size[0]
    altura_relativa = 1./size[1]
    x = (box['xmax'] + box['xmin'])/2.0
    y = (box['ymax'] + box['ymin'])/2.0
    w = abs(box['xmax'] - box['xmin'])
    h = abs(box['ymax'] - box['ymin'])
    x = x*largura_relativa
    w = w*largura_relativa
    y = y*altura_relativa
    h = h*altura_relativa

    return (x, y, w, h)

Agora definimos a função xml_to_text que ficará responsável por pegar cada anotação do nosso dataset e chamar a função convert para converter as anotações e salva-las

In [ ]:
def xml_to_txt(input_path: str, output_path: str):
    classes = {'with_mask' : '0',
                     'without_mask' : '1',
                     'mask_weared_incorrect' : '2'}

    for xml_file in glob.glob(input_path + '/*.xml'):       
        tree = ET.parse(xml_file)
        root = tree.getroot()

        txt_list = []
        for member in root.findall("object"):
            f_name = root.find("filename").text
            width, height = int(root.find('size')[0].text), int(root.find("size")[1].text)
            label = member[0].text
            box = {}
            bndbox = member.find("bndbox")
            box['xmin'] = float(bndbox.find("xmin").text)
            box['ymin'] = float(bndbox.find("ymin").text)
            box['xmax'] = float(bndbox.find("xmax").text)
            box['ymax'] = float(bndbox.find("ymax").text)

            box_yolo_format = convert((width, height), box)                     

            txt_list.append(classes.get(label) + " " + " ".join([str(l) for l in box_yolo_format]) + "\n")

        print(f"Building: {f_name.split('.')[0]}.txt")
        with open(output_path + "//" + f_name.split(".")[0] + ".txt", "w") as writer:
            for obj in txt_list:
                writer.write(obj)

Chamada da xml_to_text salvando as anotações junto das imanges

In [ ]:
xml_to_txt('/content/drive/MyDrive/faceMaskYolo/face-mask-detection/annotations', '/content/drive/MyDrive/faceMaskYolo/face-mask-detection/images')

Agora vamos dividir nosso dataset entre treino e teste. Estamos seguindo com 80% em treinamento e 20% em teste

In [ ]:
imagens = glob.glob("/content/drive/MyDrive/faceMaskYolo/face-mask-detection/images" + '/*.png')
treino, teste = train_test_split(imagens, test_size=0.2, random_state=42)


Definimos a função que ficará responsável em enviar as imagens e suas anotações para as pastas corretas

In [ ]:
import shutil
def copy_images_to_right_folder(images: list, folder: str):
  for image in images:
    shutil.copy(image, folder)
    shutil.copy(image.split('.')[0] + '.txt', folder) #linha responsável por pegar o TXT com as coordenadas

Criamos as pastas

In [ ]:
%cd /content/drive/MyDrive/faceMaskYolo
%mkdir obj
%mkdir test

/content/drive/MyDrive/faceMaskYolo


Chamamos a função copy_images_to_right_folde, passando o treino e teste, com suas respectivas pastas

In [ ]:
copy_images_to_right_folder(treino, '/content/drive/MyDrive/faceMaskYolo/obj')
copy_images_to_right_folder(teste, '/content/drive/MyDrive/faceMaskYolo/test')

Copiamos a estrutura do YOLOv4 para os nossos arquivos já que teremos que fazer algumas alterações. Após isso foi necessário fazer algumas alterações manuais, como mudar o tamanho da batch para 32, acertar os filtros aplicados nas camadas do YOLOv4, para 24, e trocar o número de classes para 3

In [ ]:
!cp /content/darknet/cfg/yolov4-custom.cfg /content/drive/MyDrive/faceMaskYolo/yolov4-obj.cfg

Agora será necessário copiar a estrutura do nosso drive para o ambiente local

In [ ]:
%cp /content/drive/MyDrive/faceMaskYolo/obj /content/darknet/data -r
%cp /content/drive/MyDrive/faceMaskYolo/test /content/darknet/data -r
%cp /content/drive/MyDrive/faceMaskYolo/obj.data /content/darknet/data 
%cp /content/drive/MyDrive/faceMaskYolo/obj.names /content/darknet/data 
%cp /content/drive/MyDrive/faceMaskYolo/yolov4-obj.cfg /content/darknet/cfg


Voltamos para dentro da pasta do darknet

In [ ]:
%cd /content/darknet

Criamos a função que pegará o caminho de cada imagem de teste e treino e salvará no arquivo txt que o YOLOv4 espera para saber onde está cada imagem de treino e teste

In [ ]:
def create_file_with_images_path_for_yolo(folder: str, subfolder: str, file_name: str, file_type: str):
  image_files = []
  path = os.path.join(folder, subfolder)
  print(path)
  os.chdir(path)
  for filename in os.listdir(os.getcwd()):
      if filename.endswith(file_type):
          image_files.append(path + "/" + filename)
  os.chdir("..")
  with open(file_name+".txt", "w") as outfile:
      for image in image_files:
          outfile.write(image)
          outfile.write("\n")
      outfile.close()
  os.chdir("..")

Chamada da função para ambos

In [ ]:
create_file_with_images_path_for_yolo('data', 'test', 'test','png')
create_file_with_images_path_for_yolo('data', 'obj', 'train','png')

data/test
data/obj


Download de uma rede pré-treinada que ajudará no treino da nossa classificação

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

# Treino

Chegamos a parte mais esperada, o treino do nosso modelo. O treino levou por volta de 16 horas.

In [ ]:
%cd /content/darknet
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg yolov4.conv.137 -dont_show -map

Após o treino, vamos criar esse função auxiliar para mostrar uma imagem

In [ ]:
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

Agora podemos ver como ficou o gráfico após o treinamento. Nele é possível ver as médias de precisão e a loss

In [ ]:
imShow('chart.png')

Para realizar uma classificação, basta mudar o tamanho do batch e as subdivisoes

In [ ]:
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov4-obj.cfg
!sed -i 's/subdivisions=32/subdivisions=1/' yolov4-obj.cfg
%cd ..

E chamar o yolov4 com os melhores pesos que alcançamos e a imagem que deseja testar

In [ ]:
!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /content/drive/MyDrive/faceMaskYolo/backup-13/yolov4-obj_best.weights /content/drive/MyDrive/faceMaskYolo/masked.jpg -thresh 0.3
imShow('predictions.jpg')